<h1 id="Librer%C3%ADas-y-variables-de-entorno">Inicializacion del Entorno<a class="anchor-link" href="#Librer%C3%ADas-y-variables-de-entorno">¶</a></h1>

In [ ]:
# UOC
# Máster de Ciencia de Datos 
# TFM - Análisis de posible cáncer de piel mediante IA
# Enrique Fernández Morales

#%pip freeze > requirements.txt

import numpy as np
import pandas as pd
import os
import cv2
import sys
import timeit
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import asarray 
from datasets import load_dataset 
from sklearn import model_selection
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from sklearn.metrics import confusion_matrix
from vit_keras import vit
from tensorflow.keras.applications import EfficientNetB0, Xception, ResNet50, ResNet152, VGG16, DenseNet201, MobileNetV2, InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.layers import Dropout
import warnings
from PIL import Image
import csv
import random

warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# GPU CUDA
# Directorio con las librerías CUDA
#os.add_dll_directory('C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.2\\bin')
# Obtención de información de la GPU dispponible
from tensorflow.python.client import device_lib
def print_info():
    print('  Versión de TensorFlow: {}'.format(tf.__version__))
    print('  GPU: {}'.format([x.physical_device_desc for x in device_lib.list_local_devices() if x.device_type=='GPU']))
    print('  Versión Cuda  -> {}'.format(tf.sysconfig.get_build_info()['cuda_version']))
    print('  Versión Cudnn -> {}\n'.format(tf.sysconfig.get_build_info()['cudnn_version']))
print_info()

In [ ]:
#Elije una configuracion (CF1, CF2, CF3, CF4, CF5) y maquela como True
#######################
CF = 'CF1'
CF1_flag = True
CF2_flag = False
CF3_flag = False
CF4_flag = False
CF5_flag = False
#######################

#Especifique los directorios necesarios
#######################
path_or = '.\\Datos\\'
path_img = path_or + f'{CF}\\imagenes\\'
input_path = path_or + f'{CF}\\salidas\\'
#######################

sys.path.append('.\\swintransformer')
from swintransformer import SwinTransformer

# Dimensiones de las imágenes para entrenar el modelo adhoc
WIDTH = 28
HEIGHT = 28

# Dimensiones de las imágenes para redes preentrenadas
WIDTH_224 = 224
HEIGHT_224 = 224

In [ ]:

#Elige que modelos entrenar y que modelos testear
#######################
train_model_cnn_adhoc_flag = False
test_model_cnn_adhoc_flag = False

train_model_vit_flag = False
test_model_vit_flag = False

train_model_ST_flag = False
test_model_ST_flag = False

train_model_EfficientNet_flag = False
test_model_EfficientNet_flag = False

train_model_xception_flag = False
test_model_xception_flag = False

train_model_resnet_flag = False
test_model_resnet_flag = False

train_model_VGG16_flag = False
test_model_VGG16_flag = False

train_model_densenet_flag = False
test_model_densenet_flag = False

train_model_mobilenet_flag = False
test_model_mobilenet_flag = False

train_model_InceptionResNetV2_flag = False
test_model_InceptionResNetV2_flag = False
#######################

#Especificar si guardar la matriz de pesos
#######################
save_matriz_pesos_flag = True
#######################

In [ ]:
if CF1_flag or CF4_flag or CF5_flag:
    # Diccionario con las classes CF1
    classes = {0: ('bcc', 'CARCINOMA'),
               1: ('df', 'DERMATOFIBROMA'),
               2: ('bkl', 'LENTIGO'),
               3: ('vasc', 'LESION_VASCULAR'),
               4: ('mel', 'MELANOMA'),
               5: ('nv', 'NEVUS'),
               6: ('akiec', 'QUERATOSIS')}

    cm_plot_classes = ['bcc', 'df', 'bkl', 'vasc', 'mel', 'nv', 'akiec']

if CF2_flag or CF3_flag:
    # Diccionario con las classes CF2
    classes = {0: ('ben', 'BENIGNO'),
               1: ('mal', 'MALIGNO')}

    cm_plot_classes = ['ben', 'mal']

<h1 id="An%C3%A1lisis-exploratoria-de-datos">Análisis exploratoria de datos<a class="anchor-link" href="#An%C3%A1lisis-exploratoria-de-datos">¶</a></h1>

In [ ]:
metadata = pd.read_csv(path_or + f'metadata_{CF}.csv', delimiter=';')

In [ ]:
metadata

In [ ]:
# Distribución de frecuencia de imágenes de cada clase - CF1
plt.figure(figsize = (12, 8))
sns.set_theme(style = 'darkgrid')
sns.countplot(data = metadata, x = 'label', order = metadata['label'].value_counts().index, palette = 'mako')
plt.xlabel('Clase', size = 15)
plt.ylabel('Cantidad', size = 15)
plt.title('Distribución de frecuencia de imágenes de cada clase', size = 20)

In [ ]:
metadata['label'].value_counts()

<h1 id="Funciones">Funciones<a class="anchor-link" href="#Funciones">¶</a></h1>

In [ ]:
# Función para la representación gráfica deL accuracy y loss de los conjuntos de entrenamiento y validación
def plot_acc_loss(histfit):
    """
    Función que representa las gráficas de accuracy y loss de los conjuntos de entrenamiento y validación.
    
    Parámetro de entrada:
    histfit: histórico del entrenamiento y validación
    """

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 4)) # 2 gráficas en la misma fila 
    fig.suptitle(f'{CF} - Accuracy y Loss del modelo')
    sns.set_theme(style = 'darkgrid')
    
    # accuracy
    ax1.plot(histfit['accuracy'])
    ax1.plot(histfit['val_accuracy'])
    ax1.set(xlabel = 'Epoch', ylabel = 'Accuracy')
    ax1.legend(['train', 'valid'], loc = 'lower right')
       
    # loss
    ax2.plot(histfit['loss'])
    ax2.plot(histfit['val_loss'])
    ax2.set(xlabel = 'Epoch', ylabel = 'Loss')
    ax2.legend(['train','valid'], loc = "upper right")

In [ ]:
# Función para representar la matriz de confusión
def plot_confusion_matrix(cm, classes, normalize = False, cmap = plt.cm.Blues):
    """
    Función que representa la matriz de confusión.
    Se puede obtener normalizada en porcentajes con: normalize = True.
    
    Parámetros de entrada:
    cm: matriz de confusión
    classes: array con las clases
    normalize: boleano para indicar si se quieren los datos normalizados o no
    cmap: paleta de colores
    
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        title = f'{CF} - Matriz de confusión normalizada en porcentajes'
    else:
        title = f'{CF} - Matriz de confusión sin normalizar'
        
    plt.figure(figsize = (10,8))
    plt.title(title, size = 18)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45, size = 13)
    plt.yticks(tick_marks, classes, size = 13)
    fmt = '.2%' if normalize else 'd'
    cm_df = pd.DataFrame(cm,
                     index = classes, 
                     columns = classes)
    sns.heatmap(cm_df, annot = True, fmt = fmt, cmap = cmap)
    plt.ylabel('Realidad', size = 15)
    plt.xlabel('Predicción', size = 15)
    plt.tight_layout()

In [ ]:
# Función para representar la curva ROC multiclase
def plot_multiclass_roc(model, X_test, y_test, classes, n_classes, figsize = (14, 7)):
    """
    Función que devuelve una representación gráfica de la curva ROC multiclase.
    
    Parámetros de entrada:
    model: modelo con el que se realiza la predicción
    X_test: características del conjunto de test
    y_test: etiquetas del conjunto de test
    n_classes: entero con el número de clases
    figsize: dimensiones del gráfico
    """
    
    # Predicción
    y_pred = model.predict(X_test)
    
    # Diccionarios
    fpr = dict()     # ratio de FP
    tpr = dict()     # ratio de TP
    roc_auc = dict() # curva ROC
    
    # Codificación one hot
    y_test_dummies = pd.get_dummies(y_test, drop_first = False).values
    
    # Iteración por todas las clases
    for i in range(n_classes):
        fpr[i], tpr[i], _ = metrics.roc_curve(y_test_dummies[:, i], y_pred[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])
        
    # ROC para cada clase
    fig, ax = plt.subplots(figsize = figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('Ratio de falsos positivos')
    ax.set_ylabel('Ratio de verdaderos positivos')
    ax.set_title(f'{CF} - CURVA ROC')
    if CF1_flag or CF4_flag or CF5_flag:
        for i in range(n_classes):
            ax.plot(fpr[i], tpr[i], label = 'Curva ROC (area = %0.2f) para la lesión cutánea %s' % (roc_auc[i], classes[i][1]))
    if CF2_flag or CF3_flag:
        ax.plot(fpr[0], tpr[0], label = 'Curva ROC (area = %0.2f) para la lesión cutánea %s' % (roc_auc[0], classes[0][1]))
        ax.plot(fpr[1], tpr[1], label = 'Curva ROC (area = %0.2f) para la lesión cutánea %s' % (roc_auc[1], classes[1][1]))
    ax.legend(loc = "best")
    ax.grid(alpha = .4)
    sns.despine()
    plt.show()


In [ ]:
# Función que entrena, valida y guarda los resultados en el diccionario histfit
def fn_train_val(histfit, model, x, y, batch_size, epochs, callbacks, x_val, y_val):
    """
    Función que entrena y valida el modelo con el conjunto de datos, número de épocas y tamaño del batch introducidos
    como parámetros de entrada.

    histfit: diccionario para guardar los resultados del entrenamiento y validación
    model: modelo a entrenar
    x: conjunto de entrenamiento
    y: etiquetas del conjunto de entrenamiento
    batch_size: tamaño del batch
    epochs: número de épocas   
    callbacks: devolución de llamadas
    x_val: 
    y_val = 

    
    Devuelve:
    histfit -- diccionario con los resultados del entrenamiento y validación
    """
    
    history = model.fit(x = x, y = y, batch_size = batch_size, epochs = epochs,
                            callbacks = callbacks, validation_data=(x_val, y_val))

    # Adición de los resultados al diccionario
    for key, value in history.history.items():
        for i in range(len(value)):
            if key != 'lr':
                histfit[key].append(value[i])
    return histfit

In [ ]:
# Función que entrena, valida y guarda los resultados en el diccionario histfit
def fn_train_val2(histfit, model, x, y, batch_size, epochs, callbacks, steps_per_epoch = None, validation_steps = None, validation_batch_size = None):
    """
    Función que entrena y valida el modelo con el conjunto de datos, número de épocas y tamaño del batch introducidos
    como parámetros de entrada.

    histfit: diccionario para guardar los resultados del entrenamiento y validación
    model: modelo a entrenar
    x: conjunto de entrenamiento
    y: etiquetas del conjunto de entrenamiento
    batch_size: tamaño del batch
    epochs: número de épocas   
    callbacks: devolución de llamadas
    steps_per_epoch: pasos por época
    validation_steps: pasos en la validación
    validation_batch_size: tamaño del batch en la validación

    
    Devuelve:
    histfit -- diccionario con los resultados del entrenamiento y validación
    """
    

    history = model.fit(x = x, validation_data = y, batch_size = batch_size,
                            epochs = epochs, callbacks = callbacks, steps_per_epoch = steps_per_epoch, 
                            validation_steps = validation_steps, validation_batch_size = validation_batch_size)
    
    
    
    # Adición de los resultados al diccionario
    for key, value in history.history.items():
        for i in range(len(value)):
            if key != 'lr':
                histfit[key].append(value[i])
    return histfit

In [ ]:
# Función de clasificación de una imagen con un modelo determinado
def class_img(model_key, image):
    """
    Función que clasifica la imagen introducida con el modelo seleccionado.
    
    Parámetros de entrada:
    model_key: clave del modelo. Diccionario: saved_models.
    image: nombre del fichero que contiene la imagen a clasificar.
    
    Devuelve la clase predicha y el porcentaje de acierto de la clasificación.
    """
    
    # Asignación del modelo
    model = saved_models[model_key]
        
    # Ruta completa de la imagen a clasificar
    img_path = image
        
    if model_key == 1:
        width = WIDTH
        height = HEIGHT
    else:
        width = WIDTH_224
        height = HEIGHT_224
        
    # Obtención de la matriz de características de la imagen
    img_arr = get_array_from_img(img_path, width, height)
    # Dimensionamiento del conjunto de datos a las dimensiones esperada por el modelo
    x = np.array(img_arr).reshape(-1, width, height, 3)
            
    # Normalización
    x = (x - np.mean(x)) / np.std(x)
        
    # Clasificación de la imagen
    result = model.predict(x)
    class_pred = max(result[0])                                        # Clase predicha
    class_ind = list(result[0]).index(class_pred)                      # Índice de la clase predicha
    class_name = classes[class_ind][0] + ' · ' + classes[class_ind][1] # Descriptivo de la clase predicha
    print('***** CLASIFICANDO CON MODELO ' + str(model_key) + '/' + str(len(saved_models)) + ' *****\n')
    print('\tResultado de la clasificación: ' + class_name)
    print('\n\tCerteza del resultado: ' + str(round(class_pred * 100, 3)) + '%\n')
    
    return class_name, round(class_pred * 100, 3)

In [ ]:
# Función para la obtención de la matriz de características de una imagen
def get_array_from_img (img_path, width, height):
    """
    Función que lee la imagen introducida como parámetro de entrada y la reescala a las dimensiones introducidas.
    Devuelve una matriz de dimensiones width x height x 3 (RGB) con las características de la imagen.
    
    Parámetros de entrada:
    img_path: ruta completa (incluido el nombre del fichero) de la imagen de entrada
    width: ancho de la imagen de salida
    height: alto de la imagen de salida
    
    Devuelve:
    img_resized: matriz con las características de la imagen de entrada con dimensiones width x height x 3
    """
    try:
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)        # lectura y conversión de formato BGR a RGB
        dim = (width, height)
        img_resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) # reescalado de la imagen
    except Exception as e:
        print(e)
    return img_resized

In [ ]:
# Función con la configuración de callbacks
def def_callbacks(e_monitor = 'val_loss', e_mode = 'min', e_patience = 5, e_verbose = 2,
                  r_monitor = 'val_accuracy', r_factor = 0.2, r_patience = 2, r_min_delta = 1e-4,
                  r_min_lr = 1e-6, r_mode = 'max', r_verbose = 2, c_filepath = 'best_model.h5', 
                  c_monitor = 'val_accuracy', c_save_best_only = True, c_save_weights_only = True,
                  c_mode = 'max', c_verbose = 2):
    """
    Función que devuelve una lista con las opciones de callbacks
    
    Devuelve:
    callbacks: lista con la configuración de la parada temprana, caida de la tasa de aprendizaje y punto de control
    
    """
    
    # Parada temprana en el caso de que la pérdida en la validación no mejore en 5 épocas
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = e_monitor,
                                                      mode = e_mode,
                                                      patience = e_patience,
                                                      verbose = e_verbose)
    
    # Caida de la tasa de aprendizaje después de 2 épocas sin mejora en el accuracy de la validación
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = r_monitor,
                                                     factor = r_factor,
                                                     patience = r_patience,
                                                     min_delta = r_min_delta,
                                                     min_lr = r_min_lr,
                                                     mode = r_mode,
                                                     verbose = r_verbose)
    
    # Guardado de los pesos del mejor modelo en función del mejor accuracy en la validación
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = c_filepath,
                                                      monitor = c_monitor,
                                                      save_best_only = c_save_best_only,
                                                      save_weights_only = c_save_weights_only,
                                                      mode = c_mode,
                                                      verbose = c_verbose)
    
    # Array con los callbacks
    callbacks = [early_stopping, checkpointer, reduce_lr]
    
    return callbacks

<h1 id="Entrenamientos">Entrenamientos<a class="anchor-link" href="#Entrenamientos">¶</a></h1>

In [ ]:
# Matriz para guardar los pesos del accuracy de cada modelo para cada clase
if CF1_flag or CF4_flag or CF5_flag:
    m1 = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 

if CF2_flag or CF3_flag:
    m1 = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)

<h2 id="Modelo-1-%C2%B7-CNN-adhoc">Modelo 1 · CNN adhoc<a class="anchor-link" href="#Modelo-1-%C2%B7-CNN-adhoc">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train_adhoc.npy')
x_test = np.load(input_path + 'x_test_adhoc.npy')
y_train = np.load(input_path + 'y_train_adhoc.npy')
y_test = np.load(input_path + 'y_test_adhoc.npy')
x_val = np.load(input_path + 'x_val_adhoc.npy')
y_val = np.load(input_path + 'y_val_adhoc.npy')

In [ ]:
# Función de creación del modelo CNN adhoc
def build_model_cnn_adhoc():
    """
    Crea una red neuronal convolucional con 4 capas convolucionales de 16, 32, 32 y 64 neuronas (kernels) de 3x3 y función de activación ReLU,
    2 capes de MaxPooling 2D de 2x2,
    1 capa Flatten que conecta la salida de la capa convolucional con una entrada de la capa densa,
    2 capas densas de 64 y 32 neuronas completamente conectadas y activación ReLU y
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """

    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same')) # capa oculta convolucional con 16 kernels de 3x3 y entrada (28, 28, 3)
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 32 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2)))                                                                      # capa de MaxPooling 2D
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))                            # capa oculta convolucional con 32 kernels de 3x3
    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 64 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))                                                    # capa de MaxPooling 2D
    model.add(Flatten())                                                                                         # capa de Flatten
    model.add(Dense(64, activation='relu'))                                                                      # capa densa completamente conectada de 64 neuronas
    model.add(Dense(32, activation='relu'))                                                                      # capa densa completamente conectada de 32 neuronas
    model.add(Dense(7, activation='softmax'))                                                                    # capa de clasificación con función Softmax para 7 clases
    
    return model

# Función de creación del modelo CNN adhoc binario
def build_model_cnn_adhoc_binary():
    """
    Crea una red neuronal convolucional con 4 capas convolucionales de 16, 32, 32 y 64 neuronas (kernels) de 3x3 y función de activación ReLU,
    2 capes de MaxPooling 2D de 2x2,
    1 capa Flatten que conecta la salida de la capa convolucional con una entrada de la capa densa,
    2 capas densas de 64 y 32 neuronas completamente conectadas y activación ReLU y
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """

    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same')) # capa oculta convolucional con 16 kernels de 3x3 y entrada (28, 28, 3)
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 32 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2)))                                                                      # capa de MaxPooling 2D
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))                            # capa oculta convolucional con 32 kernels de 3x3
    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 64 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))                                                    # capa de MaxPooling 2D
    model.add(Flatten())                                                                                         # capa de Flatten
    model.add(Dense(64, activation='relu'))                                                                      # capa densa completamente conectada de 64 neuronas
    model.add(Dense(32, activation='relu'))                                                                      # capa densa completamente conectada de 32 neuronas
    model.add(Dense(2, activation = 'softmax'))                                                                    # capa de clasificación con función sigmoid para 1 clases
    
    return model



# Función de creación del modelo CNN adhoc Dropout
def build_model_cnn_adhoc_dropout():
    """
    Crea una red neuronal convolucional con 4 capas convolucionales de 16, 32, 32 y 64 neuronas (kernels) de 3x3 y función de activación ReLU,
    2 capes de MaxPooling 2D de 2x2,
    1 capa Flatten que conecta la salida de la capa convolucional con una entrada de la capa densa,
    2 capas densas de 64 y 32 neuronas completamente conectadas y activación ReLU y
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """

    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same')) # capa oculta convolucional con 16 kernels de 3x3 y entrada (28, 28, 3)
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 32 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2)))                                                                      # capa de MaxPooling 2D
    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))                            # capa oculta convolucional con 32 kernels de 3x3
    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))                                              # capa oculta convolucional con 64 kernels de 3x3
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))                                                    # capa de MaxPooling 2D
    model.add(Flatten())                                                                                         # capa de Flatten
    model.add(Dense(64, activation='relu'))                                                                      # capa densa completamente conectada de 64 neuronas
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))                                                                      # capa densa completamente conectada de 32 neuronas
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))                                                                    # capa de clasificación con función Softmax para 7 clases
    
    return model

In [ ]:
if train_model_cnn_adhoc_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model_adhoc = build_model_cnn_adhoc()
    if CF2_flag or CF3_flag:
        model_adhoc = build_model_cnn_adhoc_binary()
    if CF5_flag:
        model_adhoc = build_model_cnn_adhoc_dropout()
    model_adhoc.summary()

In [ ]:
if train_model_cnn_adhoc_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32, 64, 128]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.001, 0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_cnn_adhoc_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history1 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo CNN adhoc...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            #train_dataset, valid_dataset = img_aumentation(x_train_adhoc, y_train_adhoc, x_test_adhoc, y_test_adhoc, batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_cnn_adhoc()
                if CF2_flag or CF3_flag:
                    model = build_model_cnn_adhoc_binary()
                if CF5_flag:
                    model = build_model_cnn_adhoc_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
                

                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val(histfit, model, x_train, y_train, batch_size, epochs,'+
                                             'callbacks, x_val, y_val)', globals = globals(), number = 1)

                # Adición de los resultados al diccionario history1
                history1['epochs'].append(epochs)
                history1['batch_size'].append(batch_size)
                history1['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history1['loss'].append(histfit['loss'][max_val_acc_idx])
                history1['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history1['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history1['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history1['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit1')
                    model.save(input_path + 'cnn_adhoc_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history1 = pd.DataFrame.from_dict(history1)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history1.to_pickle(input_path + 'df_history1')

In [ ]:
## Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history1 = pd.read_pickle(input_path + 'df_history1')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history1.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit1 = pd.read_pickle(input_path + 'df_histfit1')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit1)

In [ ]:
if test_model_cnn_adhoc_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model_adhoc = build_model_cnn_adhoc()
    if CF2_flag or CF3_flag:
        model_adhoc = build_model_cnn_adhoc_binary()
    if CF5_flag:
        model_adhoc = build_model_cnn_adhoc_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'cnn_adhoc_best_model.h5')
    model.save(input_path + 'model1.h5') # Guardado completo del modelo

    loss, acc = model.evaluate(x_test, y_test, verbose = 2)   # Evaluación del modelo con el conjunto de test

In [ ]:
# Carga del modelo entero
if CF1_flag or CF4_flag:
    model = build_model_cnn_adhoc()
if CF2_flag or CF3_flag:
    model = build_model_cnn_adhoc_binary()
if CF5_flag:
    model = build_model_cnn_adhoc_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model1.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Adición a la matriz m1 del accuracy del modelo para cada clase
for i in range(len(classes)):
    m1['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]

<h2 id="Modelo-2-%C2%B7-Red-transformer-ViT-B16">Modelo 2 · Red transformer ViT-B16<a class="anchor-link" href="#Modelo-2-%C2%B7-Red-transformer-ViT-B16">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
# Función de creación del modelo Visión Transformer ViT-B/16
def build_model_vit():
    """
    Crea un modelo de red neuronal que utiliza la red Vision Transformer con arquitectura ViT-B/16 y
    preentrenada con el dataset imagenet 21K, al que se le sustituyen las capas top de clasificación por las siguientes:
    2 capas de Flatten,
    2 capas de normalización del batch,
    1 capa densa completamente conectadad con activación gausiana y
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Importación del modelo ViT · B16 preentrenado y sin la capa de clasificación
    vit_model = vit.vit_b16(image_size = 224,
                            activation = 'softmax',
                            pretrained = True,
                            include_top = False,
                            pretrained_top = False,
                            classes = len(classes))
    
    # Creación del modelo con las nuevas capas para la clasificación
    model = tf.keras.Sequential([vit_model,
                                 keras.layers.Flatten(),                                    # capa de Flatten
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 keras.layers.Dense(11, activation = tfa.activations.gelu), # capa densa completamente contectada con activación gausiana
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 keras.layers.Dense(7, 'softmax')],                         # capa de clasificación con función Softmax para 7 clases
                                name = 'vit_16')

    return model

# Función de creación del modelo Visión Transformer ViT-B/16 binario

def build_model_vit_binary():
    """
    Crea un modelo de red neuronal que utiliza la red Vision Transformer con arquitectura ViT-B/16 y
    preentrenada con el dataset imagenet 21K, al que se le sustituyen las capas top de clasificación por las siguientes:
    2 capas de Flatten,
    2 capas de normalización del batch,
    1 capa densa completamente conectadad con activación gausiana y
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Importación del modelo ViT · B16 preentrenado y sin la capa de clasificación
    vit_model = vit.vit_b16(image_size = 224,
                            activation = 'softmax',
                            pretrained = True,
                            include_top = False,
                            pretrained_top = False,
                            classes = len(classes))
    
    # Creación del modelo con las nuevas capas para la clasificación
    model = tf.keras.Sequential([vit_model,
                                 keras.layers.Flatten(),                                    # capa de Flatten
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 keras.layers.Dense(11, activation = tfa.activations.gelu), # capa densa completamente contectada con activación gausiana
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 keras.layers.Dense(2, activation = 'softmax')],              # capa de clasificación con función sigmoid para 1 clases
                                name = 'vit_16')

    return model

# Función de creación del modelo Visión Transformer ViT-B/16 Dropout
def build_model_vit_dropout():
    """
    Crea un modelo de red neuronal que utiliza la red Vision Transformer con arquitectura ViT-B/16 y
    preentrenada con el dataset imagenet 21K, al que se le sustituyen las capas top de clasificación por las siguientes:
    2 capas de Flatten,
    2 capas de normalización del batch,
    1 capa densa completamente conectadad con activación gausiana y
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Importación del modelo ViT · B16 preentrenado y sin la capa de clasificación
    vit_model = vit.vit_b16(image_size = 224,
                            activation = 'softmax',
                            pretrained = True,
                            include_top = False,
                            pretrained_top = False,
                            classes = len(classes))
    
    # Creación del modelo con las nuevas capas para la clasificación
    model = tf.keras.Sequential([vit_model,
                                 keras.layers.Flatten(),                                    # capa de Flatten
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 Dropout(0.5),
                                 keras.layers.Dense(11, activation = tfa.activations.gelu), # capa densa completamente contectada con activación gausiana
                                 keras.layers.BatchNormalization(),                         # capa de normalización del batch
                                 Dropout(0.5),
                                 keras.layers.Dense(7, 'softmax')],                         # capa de clasificación con función Softmax para 7 clases
                                name = 'vit_16')

    return model

In [ ]:
if train_model_vit_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_vit()
    if CF2_flag or CF3_flag:
        model = build_model_vit_binary()
    if CF5_flag:
        model = build_model_vit_dropout()
    model.summary()

In [ ]:
if train_model_vit_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32, 64, 128]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.001, 0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_vit_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history2 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # CallbacKs
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo Vision Transformer ViT...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_vit()
                if CF2_flag or CF3_flag:
                    model = build_model_vit_binary()
                if CF5_flag:
                    model = build_model_vit_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate) 
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
                
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size

                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)        


                # Adición de los resultados al diccionario history2
                history2['epochs'].append(epochs)
                history2['batch_size'].append(batch_size)
                history2['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history2['loss'].append(histfit['loss'][max_val_acc_idx])
                history2['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history2['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history2['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history2['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit2')
                    model.save(input_path + 'vit_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history2 = pd.DataFrame.from_dict(history2)

    # Guardado del dataframe con todos los resultados de los entrenamientos
    df_history2.to_pickle(input_path + 'df_history2')


In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history2 = pd.read_pickle(input_path + 'df_history2')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history2.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit2 = pd.read_pickle(input_path + 'df_histfit2')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit2)

In [ ]:
if test_model_vit_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_vit()
    if CF2_flag or CF3_flag:
        model = build_model_vit_binary()
    if CF5_flag:
        model = build_model_vit_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'vit_best_model.h5')
    model.save(input_path + 'model2.h5') # Guardado completo del modelo

    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación del modelo con el conjunto de test

In [ ]:
# Carga del modelo completo
if CF1_flag or CF4_flag:
    model = build_model_vit()
if CF2_flag or CF3_flag:
    model = build_model_vit_binary()
if CF5_flag:
    model = build_model_vit_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model2.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)


In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

<h2 id="Modelo-3-%C2%B7-Swin-Transformer">Modelo 3 · Swin Transformer<a class="anchor-link" href="#Modelo-3-%C2%B7-Swin-Transformer">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
# Función de creación del modelo Swin Transformer
def build_model_SwinT():
    
    """
    Crea un modelo de red transformer que utiliza la red Swin Transformer, preentrenada con el dataset ImageNet-22K,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal transformer
    """
    
    img_adjust_layer = tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode = 'torch'), input_shape = [224, 224, 3])
    pretrained_model = SwinTransformer('swin_large_224',
                                       num_classes = 7,
                                       include_top = False,
                                       pretrained = True,
                                       use_tpu = False)
    
    model = tf.keras.Sequential([
        img_adjust_layer,
        pretrained_model,
        keras.layers.Dense(7, activation = 'softmax')
    ])
 
    return model
# Función de creación del modelo Swin Transformer binary
def build_model_SwinT_binary():
    
    """
    Crea un modelo de red transformer que utiliza la red Swin Transformer, preentrenada con el dataset ImageNet-22K,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal transformer
    """
    
    img_adjust_layer = tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode = 'torch'), input_shape = [224, 224, 3])
    pretrained_model = SwinTransformer('swin_large_224',
                                       num_classes = 7,
                                       include_top = False,
                                       pretrained = True,
                                       use_tpu = False)
    
    model = tf.keras.Sequential([
        img_adjust_layer,
        pretrained_model,
        keras.layers.Dense(2, activation = 'softmax')
    ])
 
    return model

def build_model_SwinT_dropout():
    
    """
    Crea un modelo de red transformer que utiliza la red Swin Transformer, preentrenada con el dataset ImageNet-22K,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal transformer
    """
    
    img_adjust_layer = tf.keras.layers.Lambda(lambda data: tf.keras.applications.imagenet_utils.preprocess_input(tf.cast(data, tf.float32), mode = 'torch'), input_shape = [224, 224, 3])
    pretrained_model = SwinTransformer('swin_large_224',
                                       num_classes = 7,
                                       include_top = False,
                                       pretrained = True,
                                       use_tpu = False)
    
    model = tf.keras.Sequential([
        img_adjust_layer,
        pretrained_model,
        Dropout(0.5),
        keras.layers.Dense(7, activation = 'softmax')
    ])
 
    return model

In [ ]:
if train_model_ST_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_SwinT()
    if CF2_flag or CF3_flag:
        model = build_model_SwinT_binary()
    if CF5_flag:
        model = build_model_SwinT_dropout()
    model.summary()

In [ ]:
if train_model_ST_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [2]
    # learning rates
    learning_rates = [0.00001, 0.000005]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25]
        # learning rates
        learning_rates = [0.00001, 0.000005, 0.000001]

In [ ]:
if train_model_ST_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history3 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo Swin Transformer...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_SwinT()
                if CF2_flag or CF3_flag:
                    model = build_model_SwinT_binary()
                if CF5_flag:
                    model = build_model_SwinT_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate) 
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy']) 
                            
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)     
                
                # Adición de los resultados al diccionario history3
                history3['epochs'].append(epochs)
                history3['batch_size'].append(batch_size)
                history3['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history3['loss'].append(histfit['loss'][max_val_acc_idx])
                history3['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history3['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history3['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history3['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit3')
                    model.save(input_path + 'swin_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history3 = pd.DataFrame.from_dict(history3)

    # Guardado del dataframe con todos los resultados de los entrenamientos
    df_history3.to_pickle(input_path + 'df_history3')

In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history3 = pd.read_pickle(input_path + 'df_history3')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history3.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit3 = pd.read_pickle(input_path + 'df_histfit3')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit3)

In [ ]:
if test_model_ST_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_SwinT()
    if CF2_flag or CF3_flag:
        model = build_model_SwinT_binary()
    if CF5_flag:
        model = build_model_SwinT_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'swin_best_model.h5')
    model.save(input_path + 'model3.h5') # Guardado completo del modelo

    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
# Carga del modelo completo
if CF1_flag or CF4_flag:
    model = build_model_SwinT()
if CF2_flag or CF3_flag:
    model = build_model_SwinT_binary()
if CF5_flag:
    model = build_model_SwinT_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model3.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

<h2 id="Modelo-4-%C2%B7-Red-EfficientNet-B0">Modelo 4 · Red EfficientNet B0<a class="anchor-link" href="#Modelo-4-%C2%B7-Red-EfficientNet-B0">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
def build_model_EfficientNet():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = EfficientNetB0(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                        # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_EfficientNet_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = EfficientNetB0(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                        # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_EfficientNet_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = EfficientNetB0(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                        # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
if train_model_EfficientNet_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_EfficientNet()
    if CF2_flag or CF3_flag:
        model = build_model_EfficientNet_binary()
    if CF5_flag:
        model = build_model_EfficientNet_dropout()
    model.summary()

In [ ]:
if train_model_EfficientNet_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32, 64, 128]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.001, 0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_EfficientNet_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history4 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo EfficientNet B0...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_EfficientNet()
                if CF2_flag or CF3_flag:
                    model = build_model_EfficientNet_binary()
                if CF5_flag:
                    model = build_model_EfficientNet_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)                    
                # Adición de los resultados al diccionario history4
                history4['epochs'].append(epochs)
                history4['batch_size'].append(batch_size)
                history4['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history4['loss'].append(histfit['loss'][max_val_acc_idx])
                history4['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history4['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history4['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history4['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit4')
                    model.save(input_path + 'efficientNet_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history4 = pd.DataFrame.from_dict(history4)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history4.to_pickle(input_path + 'df_history4')

In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history4 = pd.read_pickle(input_path + 'df_history4')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history4.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit4 = pd.read_pickle(input_path + 'df_histfit4')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit4)

In [ ]:
if test_model_EfficientNet_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_EfficientNet()
    if CF2_flag or CF3_flag:
        model = build_model_EfficientNet_binary()
    if CF5_flag:
        model = build_model_EfficientNet_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'efficientNet_best_model.h5')
    model.save(input_path + 'model4.h5') # Guardado completo del modelo

    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
# Carga del modelo completo
if CF1_flag or CF4_flag:
    model = build_model_EfficientNet()
if CF2_flag or CF3_flag:
    model = build_model_EfficientNet_binary()
if CF5_flag:
    model = build_model_EfficientNet_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model4.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

<h2 id="Modelo-5-%C2%B7-Red-Xception">Modelo 5 · Red Xception<a class="anchor-link" href="#Modelo-5-%C2%B7-Red-Xception">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
def build_model_Xception():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = Xception(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_Xception_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = Xception(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_Xception_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = Xception(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
if train_model_xception_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_Xception()
    if CF2_flag or CF3_flag:
        model = build_model_Xception_binary()
    if CF5_flag:
        model = build_model_Xception_dropout()
    model.summary()

In [ ]:
if train_model_xception_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32, 64, 128]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.001, 0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_xception_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history5 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo Xception...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_Xception()
                if CF2_flag or CF3_flag:
                    model = build_model_Xception_binary()
                if CF5_flag:
                    model = build_model_Xception_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])       
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)            
                # Adición de los resultados al diccionario history5
                history5['epochs'].append(epochs)
                history5['batch_size'].append(batch_size)
                history5['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history5['loss'].append(histfit['loss'][max_val_acc_idx])
                history5['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history5['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history5['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history5['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit5')
                    model.save(input_path + 'Xception_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history5 = pd.DataFrame.from_dict(history5)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history5.to_pickle(input_path + 'df_history5')

In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history5 = pd.read_pickle(input_path + 'df_history5')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history5.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit5 = pd.read_pickle(input_path + 'df_histfit5')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit5)

In [ ]:
if test_model_xception_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_Xception()
    if CF2_flag or CF3_flag:
        model = build_model_Xception_binary()
    if CF5_flag:
        model = build_model_Xception_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'Xception_best_model.h5')
    model.save(input_path + 'model5.h5') # Guardado completo del modelo

    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
if CF1_flag or CF4_flag:
    model = build_model_Xception()
if CF2_flag or CF3_flag:
    model = build_model_Xception_binary()
if CF5_flag:
    model = build_model_Xception_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model5.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

<h2 id="Modelo-6-%C2%B7-ResNet152">Modelo 6 · ResNet152<a class="anchor-link" href="#Modelo-6-%C2%B7-ResNet152">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
def build_model_ResNet152():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = ResNet152(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_ResNet152_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = ResNet152(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_ResNet152_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = ResNet152(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
if train_model_resnet_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_ResNet152()
    if CF2_flag or CF3_flag:
        model = build_model_ResNet152_binary()
    if CF5_flag:
        model = build_model_ResNet152_dropout()
    model.summary()

In [ ]:
if train_model_resnet_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32]
    # learning rates
    learning_rates = [0.0005, 0.0001, 0.00005]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.0005, 0.0001, 0.00005, 0.00001]

In [ ]:
if train_model_resnet_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history6 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo ResNet152...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_ResNet152()
                if CF2_flag or CF3_flag:
                    model = build_model_ResNet152_binary()
                if CF5_flag:
                    model = build_model_ResNet152_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])   
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)      

                # Adición de los resultados al diccionario history6
                history6['epochs'].append(epochs)
                history6['batch_size'].append(batch_size)
                history6['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history6['loss'].append(histfit['loss'][max_val_acc_idx])
                history6['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history6['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history6['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history6['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit6')
                    model.save(input_path + 'ResNet152_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history6 = pd.DataFrame.from_dict(history6)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history6.to_pickle(input_path + 'df_history6')

In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history6 = pd.read_pickle(input_path + 'df_history6')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history6.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit6 = pd.read_pickle(input_path + 'df_histfit6')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit6)

In [ ]:
if test_model_resnet_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_ResNet152()
    if CF2_flag or CF3_flag:
        model = build_model_ResNet152_binary()
    if CF5_flag:
        model = build_model_ResNet152_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'ResNet152_best_model.h5')
    model.save(input_path + 'model6.h5') # guardado del modelo completo
    
    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
# Carga del modelo completo
if CF1_flag or CF4_flag:
    model = build_model_ResNet152()
if CF2_flag or CF3_flag:
    model = build_model_ResNet152_binary()
if CF5_flag:
    model = build_model_ResNet152_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model6.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

<h2 id="Modelo-7-%C2%B7-VGG16">Modelo 7 · VGG16<a class="anchor-link" href="#Modelo-7-%C2%B7-VGG16">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
def build_model_VGG16():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = VGG16(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_VGG16_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = VGG16(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_VGG16_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = VGG16(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
if train_model_VGG16_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_VGG16()
    if CF2_flag or CF3_flag:
        model = build_model_VGG16_binary()
    if CF5_flag:
        model = build_model_VGG16_dropout()
    model.summary()

In [ ]:
if train_model_VGG16_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32, 64, 128]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.001, 0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_VGG16_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history7 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo VGG16...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_VGG16()
                if CF2_flag or CF3_flag:
                    model = build_model_VGG16_binary()
                if CF5_flag:
                    model = build_model_VGG16_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])        
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)         
                # Adición de los resultados al diccionario history7
                history7['epochs'].append(epochs)
                history7['batch_size'].append(batch_size)
                history7['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history7['loss'].append(histfit['loss'][max_val_acc_idx])
                history7['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history7['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history7['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history7['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit7')
                    model.save(input_path + 'VGG16_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history7 = pd.DataFrame.from_dict(history7)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history7.to_pickle(input_path + 'df_history7')

In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history7 = pd.read_pickle(input_path + 'df_history7')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history7.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit7 = pd.read_pickle(input_path + 'df_histfit7')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit7)

In [ ]:
if test_model_VGG16_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_VGG16()
    if CF2_flag or CF3_flag:
        model = build_model_VGG16_binary()
    if CF5_flag:
        model = build_model_VGG16_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'VGG16_best_model.h5')
    model.save(input_path + 'model7.h5') # guardado del modelo completo

    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
# Carga del modelo completo
if CF1_flag or CF4_flag:
    model = build_model_VGG16()
if CF2_flag or CF3_flag:
    model = build_model_VGG16_binary()
if CF5_flag:
    model = build_model_VGG16_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model7.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

<h2 id="Modelo-8-%C2%B7-DenseNet201">Modelo 8 · DenseNet201<a class="anchor-link" href="#Modelo-8-%C2%B7-DenseNet201">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
def build_model_DenseNet201():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = DenseNet201(include_top = False,                   # no incluir la capa de clasificación
                        input_tensor = inputs,
                        weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_DenseNet201_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = DenseNet201(include_top = False,                   # no incluir la capa de clasificación
                        input_tensor = inputs,
                        weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model


def build_model_DenseNet201_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = DenseNet201(include_top = False,                   # no incluir la capa de clasificación
                        input_tensor = inputs,
                        weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
if train_model_densenet_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_DenseNet201()
    if CF2_flag or CF3_flag:
        model = build_model_DenseNet201_binary()
    if CF5_flag:
        model = build_model_DenseNet201_dropout()
    model.summary()

In [ ]:
if train_model_densenet_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 12, 16]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.001, 0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_densenet_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history8 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo DenseNet201...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_DenseNet201()
                if CF2_flag or CF3_flag:
                    model = build_model_DenseNet201_binary()
                if CF5_flag:
                    model = build_model_DenseNet201_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])        
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)            
                # Adición de los resultados al diccionario history8
                history8['epochs'].append(epochs)
                history8['batch_size'].append(batch_size)
                history8['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history8['loss'].append(histfit['loss'][max_val_acc_idx])
                history8['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history8['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history8['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history8['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit8')
                    model.save(input_path + 'DenseNet201_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history8 = pd.DataFrame.from_dict(history8)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history8.to_pickle(input_path + 'df_history8')


In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history8 = pd.read_pickle(input_path + 'df_history8')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history8.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit8 = pd.read_pickle(input_path + 'df_histfit8')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit8)

In [ ]:
if test_model_densenet_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_DenseNet201()
    if CF2_flag or CF3_flag:
        model = build_model_DenseNet201_binary()
    if CF5_flag:
        model = build_model_DenseNet201_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'DenseNet201_best_model.h5')
    model.save(input_path + 'model8.h5') # guardado del modelo completo
    
    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
# Carga del modelo completo
if CF1_flag or CF4_flag:
    model = build_model_DenseNet201()
if CF2_flag or CF3_flag:
    model = build_model_DenseNet201_binary()
if CF5_flag:
    model = build_model_DenseNet201_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model8.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test, verbose=0)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

<h2 id="Modelo-9-%C2%B7-MobileNetV2">Modelo 9 · MobileNetV2<a class="anchor-link" href="#Modelo-9-%C2%B7-MobileNetV2">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
def build_model_MobileNetV2():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = MobileNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_MobileNetV2_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = MobileNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_MobileNetV2_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = MobileNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
if train_model_mobilenet_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_MobileNetV2()
    if CF2_flag or CF3_flag:
        model = build_model_MobileNetV2_binary()
    if CF5_flag:
        model = build_model_MobileNetV2_dropout()
    model.summary()

In [ ]:
if train_model_mobilenet_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32, 64, 128]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.001, 0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_mobilenet_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history9 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo MobileNetV2...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_MobileNetV2()
                if CF2_flag or CF3_flag:
                    model = build_model_MobileNetV2_binary()
                if CF5_flag:
                    model = build_model_MobileNetV2_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])       
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)            
                # Adición de los resultados al diccionario history9
                history9['epochs'].append(epochs)
                history9['batch_size'].append(batch_size)
                history9['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history9['loss'].append(histfit['loss'][max_val_acc_idx])
                history9['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history9['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history9['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history9['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit9')
                    model.save(input_path + 'MobileNetV2_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history9 = pd.DataFrame.from_dict(history9)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history9.to_pickle(input_path + 'df_history9')

In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history9 = pd.read_pickle(input_path + 'df_history9')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history9.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit9 = pd.read_pickle(input_path + 'df_histfit9')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit9)

In [ ]:
if test_model_mobilenet_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_MobileNetV2()
    if CF2_flag or CF3_flag:
        model = build_model_MobileNetV2_binary()
    if CF5_flag:
        model = build_model_MobileNetV2_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'MobileNetV2_best_model.h5')
    model.save(input_path + 'model9.h5') # guardado del modelo completo

    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
# Carga del modelo completo
if CF1_flag or CF4_flag:
    model = build_model_MobileNetV2()
if CF2_flag or CF3_flag:
    model = build_model_MobileNetV2_binary()
if CF5_flag:
    model = build_model_MobileNetV2_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model9.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

<h2 id="Modelo-10-%C2%B7-InceptionResNetV2">Modelo 10 · InceptionResNetV2<a class="anchor-link" href="#Modelo-10-%C2%B7-InceptionResNetV2">¶</a></h2>

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(input_path + 'x_train.npy')
x_test = np.load(input_path + 'x_test.npy')
y_train = np.load(input_path + 'y_train.npy')
y_test = np.load(input_path + 'y_test.npy')
x_val = np.load(input_path + 'x_val.npy')
y_val = np.load(input_path + 'y_val.npy')

In [ ]:
def build_model_InceptionResNetV2():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = InceptionResNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_InceptionResNetV2_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = InceptionResNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model

def build_model_InceptionResNetV2_dropout():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = InceptionResNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dropout(0.5), 
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_InceptionResNetV2()
    if CF2_flag or CF3_flag:
        model = build_model_InceptionResNetV2_binary()
    if CF5_flag:
        model = build_model_InceptionResNetV2_dropout()
    model.summary()

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32, 64, 128]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]
    if CF5_flag:
        # Hiperparámetros
        # epochs
        n_epochs = [25, 50]
        # learning rates
        learning_rates = [0.001, 0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history10 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo InceptionResNetV2...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_InceptionResNetV2()
                if CF2_flag or CF3_flag:
                    model = build_model_InceptionResNetV2_binary()
                if CF5_flag:
                    model = build_model_InceptionResNetV2_dropout()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])       
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, train_dataset, valid_dataset, batch_size, epochs,'+
                                                 'callbacks)', globals = globals(), number = 1)            
                # Adición de los resultados al diccionario history10
                history10['epochs'].append(epochs)
                history10['batch_size'].append(batch_size)
                history10['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history10['loss'].append(histfit['loss'][max_val_acc_idx])
                history10['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history10['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history10['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history10['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(input_path + 'df_histfit10')
                    model.save(input_path + 'InceptionResNetV2_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history10 = pd.DataFrame.from_dict(history10)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history10.to_pickle(input_path + 'df_history10')

In [ ]:
# Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
df_history10 = pd.read_pickle(input_path + 'df_history10')

# Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
df_history10.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
# Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
df_histfit10 = pd.read_pickle(input_path + 'df_histfit10')

# Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
plot_acc_loss(df_histfit10)

In [ ]:
if test_model_InceptionResNetV2_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_InceptionResNetV2()
    if CF2_flag or CF3_flag:
        model = build_model_InceptionResNetV2_binary()
    if CF5_flag:
        model = build_model_InceptionResNetV2_dropout()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(input_path + 'InceptionResNetV2_best_model.h5')
    model.save(input_path + 'model10.h5') # guardado del modelo completo

    loss, acc = model.evaluate(x_test, y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
# Carga del modelo completo
if CF1_flag or CF4_flag:
    model = build_model_InceptionResNetV2()
if CF2_flag or CF3_flag:
    model = build_model_InceptionResNetV2_binary()
if CF5_flag:
    model = build_model_InceptionResNetV2_dropout()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.load_weights(input_path + 'model10.h5')

In [ ]:
# Predicción con el conjunto de test
y_pred = model.predict(x_test)

In [ ]:
# Curva ROC multiclase
plot_multiclass_roc(model, x_test, y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

# Cálculo de la matriz de confusión y representación normalizada y sin normalizar
plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
#plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
# Métricas
print('***** MÉTRICAS *****')
print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
# Dataframe temporal para guardar los accuracies de cada clase de este modelo
if CF1_flag or CF4_flag or CF5_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
if CF2_flag or CF3_flag:
    m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
for i in range(len(classes)):
    m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
    
# Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
m1 = m1.append(m_temp)

In [ ]:
# Reinicio de los índices
m1 = m1.reset_index()
# Borrado de la columna index
m1 = m1.drop(['index'], axis = 1)

In [ ]:
if save_matriz_pesos_flag == True:
    # Guardado de la matriz de pesos de los modelos
    m1.to_pickle(input_path + 'm1')